In [1]:
import sagemaker
import os
import boto3

# import mlflow
import numpy as np
import json


from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.parameters import ParameterString
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.fail_step import FailStep
# from sagemaker.sklearn.processing import ScriptProcessor

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline import PipelineDefinitionConfig
from sagemaker import image_uris
from get_deployment_step import get_deployment_step
from get_wait_step import get_wait_step

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# # SageMaker Session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [3]:
def get_lambda_step(
    project: str,
    bucket_name: str,
    process_instance_count_param: int,
    process_instance_type_param: str,
    evaluation_image_uri: str,
    region: str,
    
    lambda_check_function_arn: str,
    model_package_arn: str,
    step_suffix: str
):

    print("Starting get_lambda_step")

    func = Lambda(function_arn=lambda_check_function_arn)

    # Lambda Step: Model approval status sorgulama
    return LambdaStep(
        name=f"CheckApprovalStatus{step_suffix}",
        lambda_func=func,
        inputs={"model_package_arn": model_package_arn},
        outputs=[
            LambdaOutput(output_name='Payload')
        ]
    )

In [4]:
def get_conditional_step(
    project: str,
    bucket_name: str,
    process_instance_count_param: int,
    process_instance_type_param: str,
    evaluation_image_uri: str,
    region: str,

    model_path: str,
    wait_step: ProcessingStep,
    deployment_step: ProcessingStep,
    model_package_arn: str,
    condition_step_suffix: str
):
    print("Starting conditional_step")

    step_fail = FailStep(
        name=f"{project}-fail",
        error_message="Execution failed due to ApproveStatus"
    )
    
    return ConditionStep(
        name=f"CheckIfApproved{condition_step_suffix}",
        conditions=[
            ConditionEquals(
                left=JsonGet(
                    step=wait_step,
                    s3_uri=Join(
                            on="/",
                            values=[f"s3://wildfires","model_status",f"{model_package_arn}.json"],),
                    json_path="$.ApprovalStatus"
                ),
                right="Approved"
            )
        ],
        if_steps=[deployment_step],
        else_steps=[step_fail] # wait_step
    )

In [5]:
def get_parameters() -> dict:
    process_instance_count_param = ParameterInteger(
        name="ProcessingInstanceCount",
        default_value=1
    )
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.large",
    )
    #------------------------------------------------

    #------------------------------------------------

    return {
        "process_instance_count_param": process_instance_count_param,
        "process_instance_type_param": process_instance_type_param,
    }

In [6]:
def get_pipeline(
    session: sagemaker.Session,
    parameters: dict,
    constants: dict,
    sklearn_image_uri: str
):
    pipeline_def_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

    suffix = "Initial"

    check_status_step = get_lambda_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri=sklearn_image_uri,
        region=constants["region"],

        lambda_check_function_arn='arn:aws:lambda:eu-central-1:567821811420:function:LambdaWildfireCheckStatus',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1',
        step_suffix=suffix
    )

    wait_step = get_wait_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri=sklearn_image_uri,
        region=constants["region"],
        
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    )

    deployment_step = get_deployment_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri=sklearn_image_uri,
        region=constants["region"],
        
        model_prefix='models',
        model_filename='model_resnet18.tar.gz',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    )

    conditional_step = get_conditional_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri=sklearn_image_uri,
        region=constants["region"],
        
        model_path='',
        deployment_step=deployment_step,
        wait_step=wait_step,
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1',
        condition_step_suffix=suffix
    )

    # ------------------------------------------------

    wait_step.add_depends_on([check_status_step])
    conditional_step.add_depends_on([wait_step])

    return Pipeline(
        name=f"{constants['project']}-pipeline",
        parameters=[parameters[key] for key in parameters],
        pipeline_definition_config=pipeline_def_config,
        steps=[
            # processor_step,
            # evaluation_step,
            # check_status_step,
            # wait_step,
            # conditional_step
            deployment_step
        ],
    )

In [ ]:
print(os.getcwd())

parameters = get_parameters()

constants = {
    "region": "eu-central-1",
    "project": "wildfire-dumm-8",
    "bucket_name": "wildfires",
    "sklearn_image_uri_version": "1.2-1",
}

session = sagemaker.Session(boto3.Session(region_name=constants["region"]))

sklearn_image_uri = image_uris.retrieve(
    framework="sklearn",
    region=constants["region"],
    version=constants["sklearn_image_uri_version"],
)

pipeline = get_pipeline(
    session=session,
    parameters=parameters,
    constants=constants,
    sklearn_image_uri=sklearn_image_uri,
)

pipeline.upsert(role_arn=sagemaker.get_execution_role())

/home/sagemaker-user/Steps/Job 5 Manuel App and Deployment


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


Starting get_lambda_step
